In [40]:
import duckdb
import pandas as pd

In [62]:
sql_query = """
            SELECT
                violation_code
                ,definition
                ,manhattan_96th_st_below
                all_other_areas
            FROM
                parking_violation_codes
            """

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,violation_code,definition,all_other_areas
0,1,FAILURE TO DISPLAY BUS PERMIT,515
1,2,NO OPERATOR NAM/ADD/PH DISPLAY,515
2,3,UNAUTHORIZED PASSENGER PICK-UP,515
3,4,BUS PARKING IN LOWER MANHATTAN,115
4,5,BUS LANE VIOLATION,250
...,...,...,...
92,93,REMOVE/REPLACE FLAT TIRE,65
93,96,RAILROAD CROSSING,95
94,97,VACANT LOT,65
95,98,OBSTRUCTING DRIVEWAY,95


In [56]:
sql_query_import_1 = """
                    CREATE OR REPLACE TABLE parking_violation_codes AS
                    SELECT
                        "CODE" AS violation_code,
                        "DEFINITION" AS definition,
                        "Manhattan 96th St. & below" AS manhattan_96th_st_below,
                        "All Other Areas" AS all_other_areas
                    FROM read_csv_auto(
                        'data/dof_parking_violation_codes.csv',
                        columns={'CODE': 'INTEGER', 
                                 'DEFINITION': 'STRING', 
                                 'Manhattan 96th St. & below': 'INTEGER', 
                                 'All Other Areas': 'INTEGER'},
                        header=True,
                        normalize_names=False
                    );
                    """

sql_query_import_2 = """
                    CREATE OR REPLACE TABLE parking_violation_2023 AS
                    SELECT *
                    FROM read_csv_auto(
                        'data/parking_violations_issued_fiscal_year_2023_sample.csv',
                        normalize_names=True
                    )
                    """

with duckdb.connect('data/nyc_parking_violations.db') as con:
    con.sql(sql_query_import_1)
    con.sql(sql_query_import_2)

In [58]:
sql_query_schema = """
                   PRAGMA table_info('parking_violation_codes')
                   """

with duckdb.connect('data/nyc_parking_violations.db') as con:
    print(con.sql(sql_query_schema).df())

   cid                     name     type  notnull dflt_value     pk
0    0           violation_code  INTEGER    False       None  False
1    1               definition  VARCHAR    False       None  False
2    2  manhattan_96th_st_below  INTEGER    False       None  False
3    3          all_other_areas  INTEGER    False       None  False
